# Import appropriate packages and set analysis options.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

print_all_output = True
InteractiveShell.ast_node_interactivity = 'all' if print_all_output else 'last_expr'

# Define convenient variables and functions.

In [2]:
TRAINING_DATA_PATH = "./Data/train.csv"

# Load the file given by 'path' into a Pandas DataFrame.
def load_data(path):
    df = pd.read_csv(path, header=0, index_col='Id')
    print("Loaded data dimensions: ", df.shape[0], "rows, ", df.shape[1], "columns")
    return df

# Print all columns which contain NaN values.
def print_if_nan(df, columns, col_type):
    nan_cols = []
    nan_counts = []
    for col in np.sort(columns):
        num_nan = np.sum(x_train[col].isnull())
        if (num_nan > 0):
            nan_cols.append(col)
            nan_counts.append(num_nan)
    print("\n", len(nan_cols), " ", col_type, "-type columns with NaN values.", sep='')
    for col, count in zip(nan_cols, nan_counts):
          print(col, "\t", count)

# Load and preview the housing training data.

In [3]:
df = load_data(TRAINING_DATA_PATH)

Loaded data dimensions:  1460 rows,  80 columns


In [4]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


# Clean and process the training data.

In [6]:
# Partition data into features (X-data) and targets (Y-data).
x_train = df.iloc[:,:79]
y_train = df.iloc[:,79]

In [7]:
# Determine which features have missing values.
int_cols = x_train.select_dtypes(include=['int']).columns
print_if_nan(x_train, int_cols, "int")

float_cols = x_train.select_dtypes(include=['float']).columns
print_if_nan(x_train, float_cols, "float")

string_cols = x_train.select_dtypes(include=['dtype']).columns
print_if_nan(x_train, string_cols, "string")


0 int-type columns with NaN values.

3 float-type columns with NaN values.
GarageYrBlt 	 81
LotFrontage 	 259
MasVnrArea 	 8

16 string-type columns with NaN values.
Alley 	 1369
BsmtCond 	 37
BsmtExposure 	 38
BsmtFinType1 	 37
BsmtFinType2 	 38
BsmtQual 	 37
Electrical 	 1
Fence 	 1179
FireplaceQu 	 690
GarageCond 	 81
GarageFinish 	 81
GarageQual 	 81
GarageType 	 81
MasVnrType 	 8
MiscFeature 	 1406
PoolQC 	 1453
